In [59]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import math

#%matplotlib inline

In [60]:
events = pd.read_csv('data/calcularis_small_events.csv', encoding='latin', low_memory=False)
subtasks = pd.read_csv('data/calcularis_small_subtasks.csv', encoding='latin', low_memory=False)
users = pd.read_csv('data/calcularis_small_users.csv', encoding='latin', low_memory=False)

In [61]:
events.head(15)

,event_id,user_id,mode,game_name,learning_time_ms,number_range,start,end,skill_id,type
0,0,1,NORMAL,Subitizing,"8,835.00000",R10,2022-11-02T08:39:12.355Z,2022-11-02T08:39:25.130Z,1.00000,task
1,1,1,NORMAL,Conversion,"21,167.00000",R10,2022-11-11T10:26:27.893Z,2022-11-11T10:26:49.260Z,4.00000,task
2,2,1,NORMAL,Conversion,"11,182.00000",R10,2022-11-18T10:34:01.044Z,2022-11-18T10:34:12.423Z,7.00000,task
3,3,1,NORMAL,Landing,"6,823.00000",R10,2022-11-25T10:32:43.428Z,2022-11-25T10:32:56.986Z,19.00000,task
4,4,1,END_OF_NR,Conversion,"9,107.00000",R10,2022-12-02T10:44:40.555Z,2022-12-02T10:44:49.874Z,7.00000,task
5,5,1,END_OF_NR,Conversion,"10,703.00000",R10,2022-12-09T10:12:16.068Z,2022-12-09T10:12:26.984Z,4.00000,task
6,6,1,NORMAL,Comparison,"1,383.00000",R20,2022-12-16T10:25:42.441Z,2022-12-16T10:25:45.508Z,33.00000,task
7,7,1,END_OF_NR,Landing,"6,052.00000",R20,2023-01-20T10:13:41.496Z,2023-01-20T10:13:49.096Z,50.00000,task
8,8,1,END_OF_NR,Landing,"6,055.00000",R20,2023-01-27T10:18:17.427Z,2023-01-27T10:18:26.356Z,49.00000,task
9,9,1,END_OF_NR,Estimation on Number Line,"10,541.00000",R20,2023-02-03T10:20:25.581Z,2023-02-03T10:20:38.687Z,39.00000,task


In [62]:
subtasks.head(15)

,subtask_id,event_id,user_id,aim,answer,answerMode,availableNumbers,correct,correctAnswerObject,correctNumber,...,startPosition,subtask_finished_timestamp,target,timeoutInSeconds,timeoutInSteps,type,upperBound,divisor,orderIndependent,step
0,0,0,1,NaN,4,NaN,NaN,True,4,4.00000,...,NaN,2022-11-02T08:39:24.930Z,NaN,NaN,NaN,ConciseSubitizingTaskDescription,NaN,NaN,NaN,NaN
1,1,0,1,NaN,1,NaN,NaN,True,NaN,NaN,...,NaN,2022-11-02T08:39:24.930Z,NaN,0.00000,2.00000,ConciseTimeoutDescription,NaN,NaN,NaN,NaN
2,2,1,1,NaN,3,NaN,NaN,True,3,NaN,...,NaN,2022-11-11T10:26:49.007Z,NaN,NaN,NaN,ConciseConversionTaskDescription,NaN,NaN,NaN,NaN
3,3,2,1,NaN,5,NaN,NaN,True,5,NaN,...,NaN,2022-11-18T10:34:12.191Z,NaN,NaN,NaN,ConciseConversionTaskDescription,NaN,NaN,NaN,NaN
4,4,3,1,3.00000,"{'a': 2, 'b': 2.0402703}",NaN,NaN,False,"{'a': 3, 'b': 3.0}",NaN,...,0.50000,2022-11-25T10:32:56.805Z,NaN,NaN,NaN,ConciseLandingTaskDescription,3.50000,NaN,NaN,NaN
5,5,4,1,NaN,9,NaN,NaN,True,9,NaN,...,NaN,2022-12-02T10:44:49.621Z,NaN,NaN,NaN,ConciseConversionTaskDescription,NaN,NaN,NaN,NaN
6,6,5,1,NaN,7,NaN,NaN,False,9,NaN,...,NaN,2022-12-09T10:12:26.729Z,NaN,NaN,NaN,ConciseConversionTaskDescription,NaN,NaN,NaN,NaN
7,7,6,1,NaN,16,NaN,NaN,True,16,NaN,...,NaN,2022-12-16T10:25:45.293Z,NaN,NaN,NaN,ConciseSetComparisonTaskDescription,NaN,NaN,NaN,NaN
8,8,6,1,NaN,1.3659,NaN,NaN,True,NaN,NaN,...,NaN,2022-12-16T10:25:45.293Z,NaN,4.00000,0.00000,ConciseTimeoutDescription,NaN,NaN,NaN,NaN
9,9,7,1,4.00000,"{'a': 4, 'b': 4.148817}",NaN,NaN,True,"{'a': 4, 'b': 4.0}",NaN,...,0.50000,2023-01-20T10:13:48.879Z,NaN,NaN,NaN,ConciseLandingTaskDescription,5.00000,NaN,NaN,NaN


In [63]:
users.head(15)

,user_id,learning_time_ms,start,end,logged_in_time_ms,language,country
0,1,14032710,2022-11-02T08:37:56.549Z,2023-02-09T11:08:02.599Z,22151340,de,CH
1,2,16268350,2022-09-07T07:53:38.865Z,2023-02-09T08:39:14.692Z,85421273,nl,NL
2,3,8012030,2021-09-27T07:45:51.806Z,2022-01-13T12:14:09.565Z,16651482,de,DE
3,4,1414421,2019-11-12T12:18:15.724Z,2020-10-02T09:20:28.798Z,4561768,de,CH
4,5,17502108,2022-04-26T11:38:44.114Z,2022-08-29T15:52:11.087Z,25601470,de,CH
5,6,8353125,2022-09-02T07:27:20.675Z,2023-02-01T10:35:15.218Z,19249399,nl,NL
6,7,7226229,2015-03-19T18:47:32.621Z,2015-05-29T18:17:32.889Z,9225357,NaN,NaN
7,8,64629753,2021-08-23T09:04:32.478Z,2022-06-20T12:13:06.081Z,125236245,de,CH
8,9,7647781,2019-08-20T17:02:06.155Z,2020-03-08T14:08:06.321Z,9089877,de,DE
9,10,22569161,2017-08-30T06:27:19.365Z,2021-07-08T14:09:17.450Z,33642991,de,NaN


In [64]:
# extract game_name and number_range from the events dataframe to match the subtasks dataframe (given the event_id)
game_name = subtasks.merge(events[['event_id', 'game_name']], on='event_id', how='left').game_name
number_range = subtasks.merge(events[['event_id', 'number_range']], on='event_id', how='left').number_range

#split the game_name into skills
skill = game_name.apply(lambda x: 'Number representation' 
                        if x in ['Subitizing', 'Conversion', 'Estimation', 'Estimation on Number Line', 'Scale: Conversion', 'Landing'] 
                        else 'Number comparison/manipulation' if x in ['Comparison', 'Secret Number', 'Distance', 'Scale: Composition', 'Order'] 
                        else 'Addition/Substraction' if x in ['Scale: Decomposition', 'Scale: Subtraction', 'Scale: Subtraction by Tens and Ones', 'Sliderule', 'Plus-minus', 'Difference', 'Completion', 'Calculator'] 
                        else 'Multiplication/Division' if x in ['Write as multiplication', 'Shelf: Jump Ahead', 'Shelf: given height', 'Shelf: random height', 'Distribution', 'Calculator: Multiplication', 'Calculator: Multiplication with ?', 'Calculator: Division', 'Jump backwards', 'Series'] 
                        else 'Other')

# concatenate skills and number range and add it to the subtasks dataframe
subtasks['skill_name'] = skill + ' ' + number_range
subtasks

,subtask_id,event_id,user_id,aim,answer,answerMode,availableNumbers,correct,correctAnswerObject,correctNumber,...,subtask_finished_timestamp,target,timeoutInSeconds,timeoutInSteps,type,upperBound,divisor,orderIndependent,step,skill_name
0,0,0,1,NaN,4,NaN,NaN,True,4,4.00000,...,2022-11-02T08:39:24.930Z,NaN,NaN,NaN,ConciseSubitizingTaskDescription,NaN,NaN,NaN,NaN,Number representation R10
1,1,0,1,NaN,1,NaN,NaN,True,NaN,NaN,...,2022-11-02T08:39:24.930Z,NaN,0.00000,2.00000,ConciseTimeoutDescription,NaN,NaN,NaN,NaN,Number representation R10
2,2,1,1,NaN,3,NaN,NaN,True,3,NaN,...,2022-11-11T10:26:49.007Z,NaN,NaN,NaN,ConciseConversionTaskDescription,NaN,NaN,NaN,NaN,Number representation R10
3,3,2,1,NaN,5,NaN,NaN,True,5,NaN,...,2022-11-18T10:34:12.191Z,NaN,NaN,NaN,ConciseConversionTaskDescription,NaN,NaN,NaN,NaN,Number representation R10
4,4,3,1,3.00000,"{'a': 2, 'b': 2.0402703}",NaN,NaN,False,"{'a': 3, 'b': 3.0}",NaN,...,2022-11-25T10:32:56.805Z,NaN,NaN,NaN,ConciseLandingTaskDescription,3.50000,NaN,NaN,NaN,Number representation R10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55042,55042,37415,998,46.00000,"{'a': 47, 'b': 47.33128}",NaN,NaN,True,"{'a': 46, 'b': 46.0}",NaN,...,2021-01-06T14:14:46.133Z,NaN,NaN,NaN,ConciseLandingTaskDescription,50.00000,NaN,NaN,NaN,Number representation R100
55043,55043,37417,1000,NaN,1,NaN,NaN,False,3,3.00000,...,2019-09-30T10:04:58.024Z,NaN,NaN,NaN,ConciseSubitizingTaskDescription,NaN,NaN,NaN,NaN,Number representation R10
55044,55044,37417,1000,NaN,2,NaN,NaN,True,NaN,NaN,...,2019-09-30T10:04:58.024Z,NaN,0.00000,2.00000,ConciseTimeoutDescription,NaN,NaN,NaN,NaN,Number representation R10
55045,55045,37418,1000,NaN,3,NaN,NaN,True,3,NaN,...,2020-01-20T10:03:51.556Z,NaN,NaN,NaN,ConciseConversionTaskDescription,NaN,NaN,NaN,NaN,Number representation R10


# BKT Model

In [65]:
from pyBKT.models import Model

In [66]:
model = Model(seed=0, defaults={'order_id' : 'subtask_id'})
model.fit(data=subtasks, 
          multilearn = 'user_id',
          multigs = 'user_id',
          forgets = True)

In [67]:
params = model.params()
params

value
skill                       param   class          
Number representation R10   prior   default 0.33148
                            learns  1       0.40508
                                    10      0.34954
                                    100     0.37789
                                    1000    0.61743
...                                             ...
Multiplication/Division R10 forgets 952     0.04748
                                    953     0.04748
                                    964     0.04748
                                    966     0.04748
                                    980     0.04748

[21224 rows x 1 columns]

In [68]:
b = model.predict(data=subtasks)

In [73]:
a = subtasks.groupby(['user_id', 'skill_name']).last()
display(a[['correct', 'correct_predictions', 'state_predictions']])
b[['user_id', 'skill_name', 'correct', 'correct_predictions', 'state_predictions']].head(40)
#model.predict(data=subtasks)

correct  correct_predictions   
user_id skill_name                                                         
1       Number comparison/manipulation R20        0              0.50293  \
        Number representation R10                 0              0.69992   
        Number representation R20                 1              1.00000   
2       Addition/Substraction R10                 1              1.00000   
        Number comparison/manipulation R10        0              0.76222   
...                                             ...                  ...   
998     Number comparison/manipulation R20        1              1.00000   
        Number representation R10                 0              0.69908   
        Number representation R100                1              0.50016   
        Number representation R20                 1              0.82374   
1000    Number representation R10                 1              0.83457   

                                            state_predictions  
user_id skill_name                                             
1       Number comparison/manipulation R20            0.12038  
        Number representation R10                     0.43787  
        Number representation R20                     0.98187  
2       Addition/Substraction R10                     0.89482  
        Number comparison/manipulation R10            0.24576  
...                                                       ...  
998     Number comparison/manipulation R20            0.02208  
        Number representation R10                     0.43476  
        Number representation R100                    0.00135  
        Number representation R20                     0.98187  
1000    Number representation R10                     0.67970  

[5302 rows x 3 columns]

,user_id,skill_name,correct,correct_predictions,state_predictions
0,1,Number representation R10,1,0.64629,0.33148
1,1,Number representation R10,1,0.69766,0.43339
2,1,Number representation R10,1,0.70067,0.43937
3,1,Number representation R10,1,0.70083,0.43969
4,1,Number representation R10,0,0.70084,0.43971
5,1,Number representation R10,1,0.68409,0.40647
6,1,Number representation R10,0,0.69992,0.43787
7,1,Number comparison/manipulation R20,1,0.85016,0.73485
8,1,Number comparison/manipulation R20,1,0.57287,0.24414
9,1,Number representation R20,1,1.00000,0.58298
